<a href="https://colab.research.google.com/github/kayla-jackson/spatial-modeling/blob/test-exploratory-sims/notebooks/mcmc_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Colab setup ------------------
import os, sys, subprocess
if "google.colab" in sys.modules:
    cmd = "pip install --upgrade iqplot colorcet bebi103 arviz cmdstanpy"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    import cmdstanpy; cmdstanpy.install_cmdstan()
# ------------------------------

KeyboardInterrupt: ignored

In [2]:
# Clone github repo
!git clone --branch test-exploratory-sims https://github.com/kayla-jackson/spatial-modeling.git

Cloning into 'spatial-modeling'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 98 (delta 39), reused 61 (delta 18), pack-reused 0
Receiving objects: 100% (98/98), 5.82 MiB | 5.84 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [3]:
# Setup directories
repo_dir = "./spatial-modeling/"
data_dir = os.path.join(repo_dir, "data")
stan_dir = os.path.join(repo_dir, "inst/stan")

# Load libraries

In [4]:
import numpy as np
import pandas as pd
import scipy.special
import scipy.stats as st

import cmdstanpy
import arviz as az

# plotting packages
import iqplot
import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

# Parameter Estimation with Markov Chain Monte Carlo (MCMC)

The code below demonstrates how to perform parameter estimation using MCMC. The model is defined in the file `inst/stan/car.stan`.

First, we have to compile the model.

In [5]:
fn = os.path.join(stan_dir, "car_prob.stan")

sm = cmdstanpy.CmdStanModel(stan_file=fn)

06:26:12 - cmdstanpy - INFO - compiling stan file /content/spatial-modeling/inst/stan/car_prob.stan to exe file /content/spatial-modeling/inst/stan/car_prob
INFO:cmdstanpy:compiling stan file /content/spatial-modeling/inst/stan/car_prob.stan to exe file /content/spatial-modeling/inst/stan/car_prob
DEBUG:cmdstanpy:cmd: make STANCFLAGS+=--filename-in-msg=car_prob.stan /content/spatial-modeling/inst/stan/car_prob
cwd: /root/.cmdstan/cmdstan-2.33.1
DEBUG:cmdstanpy:Console output:

--- Translating Stan model to C++ code ---
bin/stanc --filename-in-msg=car_prob.stan --o=/content/spatial-modeling/inst/stan/car_prob.hpp /content/spatial-modeling/inst/stan/car_prob.stan

--- Compiling, linking C++ code ---
g++ -std=c++1y -pthread -D_REENTRANT -Wno-sign-compare -Wno-ignored-attributes      -I stan/lib/stan_math/lib/tbb_2020.3/include    -O3 -I src -I stan/src -I stan/lib/rapidjson_1.1.0/ -I lib/CLI11-1.9.1/ -I stan/lib/stan_math/ -I stan/lib/stan_math/lib/eigen_3.4.0 -I stan/lib/stan_math/lib/bo

Next, we specify the data and let Stan do the rest of the work. The paramers that you need to specify are as follows:


* `count_n`: The number of points you will use to estimate parameters.
* `counts`: The data you will use for fitting. Should be an array of size `count_n`.

* `rates`: Represents the probability of capture at each grid point. Should be between `$0$` and `$1$`. Should be an array of size `count_n`.


* `bs_mu`: For now, set to 0
* `mu_mu`: For now, set to 0

* `bs_var`: For now, set to 1
* `mu_var`: For now, set to 5



In [32]:
# Importing the data - 100 genes, 64 valid positions
COUNT_N = 6400

df = pd.read_csv(os.path.join(data_dir, 'counts.csv'))

df = df[~df['border']] # Filter out locations where border = True

# Create counts array
# ordered version: gene_labels = [f'gene_{i}' for i in range(1, 101)]
# gene_counts = df.iloc[:, -100:][gene_labels].values.flatten()

gene_counts = df.iloc[:, -100:].values.flatten()
true_p = np.ravel([[p]*100 for p in df['theta'].values])

'''
W = np.array([
    0, 1, 1, 0,
    1, 0, 0, 1,
    1, 0, 0, 1,
    0, 1, 1, 0
])
'''

data = dict(
    count_n=COUNT_N,
    counts=gene_counts,
    rates=true_p,

    bs_mu=0,
    bs_var=1,
    mu_mu=0,
    mu_var=5,
)

In [33]:
# Perform sampling with Stan
samples = sm.sample(
    data=data,
    chains=4,
    iter_sampling=1000,
)

# Convert to ArviZ InferenceData instance
samples = az.from_cmdstanpy(posterior=samples)

DEBUG:cmdstanpy:cmd: /content/spatial-modeling/inst/stan/car_prob info
cwd: None
DEBUG:cmdstanpy:input tempfile: /tmp/tmpd__drkl7/sjdlm0dl.json
06:59:05 - cmdstanpy - INFO - CmdStan start processing
INFO:cmdstanpy:CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:CmdStan args: ['/content/spatial-modeling/inst/stan/car_prob', 'id=1', 'random', 'seed=6056', 'data', 'file=/tmp/tmpd__drkl7/sjdlm0dl.json', 'output', 'file=/tmp/tmpd__drkl7/car_probg1xlq6nl/car_prob-20231017065905_1.csv', 'method=sample', 'num_samples=1000', 'algorithm=hmc', 'adapt', 'engaged=1']
DEBUG:cmdstanpy:idx 1
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:CmdStan args: ['/content/spatial-modeling/inst/stan/car_prob', 'id=2', 'random', 'seed=6056', 'data', 'file=/tmp/tmpd__drkl7/sjdlm0dl.json', 'output', 'file=/tmp/tmpd__drkl7/car_probg1xlq6nl/car_prob-20231017065905_2.csv', 'method=sample', 'num_samples=1000', 'algorithm=hmc', 'adapt', 'engaged=1']
DEBUG:cmdstanpy:idx 2
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:CmdStan args: ['/content/spatial-modeling/inst/stan/car_prob', 'id=3', 'random', 'seed=6056', 'data', 'file=/tmp/tmpd__drkl7/sjdlm0dl.json', 'ou

06:59:55 - cmdstanpy - INFO - CmdStan done processing.
INFO:cmdstanpy:CmdStan done processing.
DEBUG:cmdstanpy:runset
RunSet: chains=4, chain_ids=[1, 2, 3, 4], num_processes=4
 cmd (chain 1):
	['/content/spatial-modeling/inst/stan/car_prob', 'id=1', 'random', 'seed=6056', 'data', 'file=/tmp/tmpd__drkl7/sjdlm0dl.json', 'output', 'file=/tmp/tmpd__drkl7/car_probg1xlq6nl/car_prob-20231017065905_1.csv', 'method=sample', 'num_samples=1000', 'algorithm=hmc', 'adapt', 'engaged=1']
 retcodes=[0, 0, 0, 0]
 per-chain output files (showing chain 1 only):
 csv_file:
	/tmp/tmpd__drkl7/car_probg1xlq6nl/car_prob-20231017065905_1.csv
 console_msgs (if any):
	/tmp/tmpd__drkl7/car_probg1xlq6nl/car_prob-20231017065905_0-stdout.txt
DEBUG:cmdstanpy:Chain 1 console:
method = sample (Default)
  sample
    num_samples = 1000 (Default)
    num_warmup = 1000 (Default)
    save_warmup = 0 (Default)
    thin = 1 (Default)
    adapt
      engaged = 1 (Default)
      gamma = 0.050000000000000003 (Default)
      delt

AttributeError: ignored

Take a quick look at the posterior distribution

In [26]:
samples.posterior

AttributeError: ignored

In [ ]:
# Convert posterior samples to dataframe for plotting
df_mcmc = samples.posterior.to_dataframe()

# Take a look
df_mcmc.head()

In [25]:
# Some plots of the parameters
plots = [
    iqplot.histogram(df_mcmc, q=param, rug=False)
    for param in ["mu", "burst_size"]
]

bokeh.io.show(bokeh.layouts.gridplot(plots, ncols=2))

NameError: ignored

Isolate the parameters as an  `np.array()`

In [ ]:
mu_vals = df_mcmc.mu.values.flatten()
bs_vals = df_mcmc.burst_size.values.flatten()

The vector `mu_vals` (or `bs_vals`) contain many estimates for the parameters `mu` or `burst size`, but how do you know which one to pick? You can simply take the mean of these vectors (e.g. `np.mean(mu_vals)) and use that or the best estimate, but if the distribution is skewed like it is for burst size, you may want to choose some other quantile instead of the mean.

You can explore this from plotting.

In [ ]:
# np.quantile(mu_vals, [0, 0.025, 0.5, 0.95, 1])
np.quantile(bs_vals, [0, 0.025, 0.5, 0.95, 1])

In [ ]:
# save the parameter estimates for the current "rates"
best_mu = np.mean(mu_vals)
best_bs = np.quantile(bs_vals, 0.5)

Repeat the steps above, changing the input `rates` at each iteration. Keep track of the `best_mu` and `best_bs`. How do these values change for each new set of rates?

In [13]:
# Generating different rates
GRID_SIZE = 10 # dimensions of spatial grid
NUM_SAMPLES = 1000


# Helper functions to generate a sample from MVN(0, sigma)

def adjacency_matrix(A):
    '''Returns the adjacency matrix W from A, a NumPy array'''
    rows, cols = len(A), len(A[0])
    locs = rows * cols # number of spatial locations on the grid
    W = np.zeros((locs, locs), dtype=int)

    # Iterate through each element in A to determine number of neighbors
    for r in range(rows):
        for c in range(cols):
            for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
                if 0 <= r + dr < rows and 0 <= c + dc < cols:
                    W[r * cols + c][(r + dr) * cols + c + dc] = 1

    return W

def sum_adjacency_matrix(W):
    '''Outputs matrix D with same dimensions as W;
    Sums the number of neighbors of the original points'''
    return np.diag(np.sum(W, axis=1))

def compute_sigma(A, alpha, tau):
    '''Outputs the covariance matrix termed sigma from original matrix A'''
    W = adjacency_matrix(A)
    D = sum_adjacency_matrix(W)

    Q = tau * (D - alpha * W) # the formula
    return np.linalg.inv(Q)

def MVN_sample(sigma, num_samples):
    '''Returns a sample from a multivariate normal distribution
    where sigma is a covariance matrix'''
    zero_vector = np.zeros(len(sigma))
    samples = np.random.multivariate_normal(zero_vector, sigma, num_samples)
    return samples

def inverse_logit(M):
    '''Transforms a matrix of values into the range (0,1)'''
    return 1 / (1 + np.exp(-M))

# Generating the probability matrix
def generate():
    A = np.zeros((GRID_SIZE, GRID_SIZE), dtype=int)
    W = adjacency_matrix(A)
    D = sum_adjacency_matrix(W)
    sigma = compute_sigma(A, 0.99, 1)

    samples = MVN_sample(sigma, NUM_SAMPLES)
    return inverse_logit(samples)

In [14]:
# something might be off with the grid size here
p_i = [generate() for x in range(10)]

print(p_i)

[array([[0.40632479, 0.49009884, 0.69785571, ..., 0.53095697, 0.29949125,
        0.57820621],
       [0.10682142, 0.25752427, 0.40654043, ..., 0.3922786 , 0.58120362,
        0.18953215],
       [0.46546016, 0.39776064, 0.36826243, ..., 0.25117777, 0.27182522,
        0.27978582],
       ...,
       [0.11932688, 0.26983823, 0.24579547, ..., 0.64764365, 0.54947655,
        0.52125175],
       [0.05681136, 0.26631368, 0.28859132, ..., 0.61245367, 0.34702894,
        0.4529401 ],
       [0.50991484, 0.427491  , 0.38296651, ..., 0.6142535 , 0.67330124,
        0.70590274]]), array([[0.79297366, 0.69806685, 0.84305732, ..., 0.38781732, 0.73755279,
        0.25823321],
       [0.39488411, 0.2264663 , 0.38754491, ..., 0.93592407, 0.6487628 ,
        0.91274787],
       [0.26421212, 0.34994183, 0.37854291, ..., 0.53393341, 0.51603356,
        0.6398354 ],
       ...,
       [0.53757793, 0.45502373, 0.73526997, ..., 0.78023038, 0.83478234,
        0.81175571],
       [0.66117037, 0.65033924, 0

In [17]:
results = {}

COUNT_N = 6400

for i, rates in enumerate(p_i):
    data = dict(
        count_n=COUNT_N,
        counts=gene_counts,
        rates=rates,

        bs_mu=0,
        bs_var=1,
        mu_mu=0,
        mu_var=5,
    )

    # Perform sampling with Stan
    samples = sm.sample(
        data=data,
        chains=4,
        iter_sampling=1000,
    )

    # Convert to ArviZ InferenceData instance
    samples = az.from_cmdstanpy(posterior=samples)

    df_mcmc = samples.posterior.to_dataframe()

    # Calculate parameter estimates
    mu_vals = df_mcmc.mu.values.flatten()
    bs_vals = df_mcmc.burst_size.values.flatten()

    # Store the parameter estimates for these rates
    results[f'Rates {i}'] = {
        'best_mu': np.mean(mu_vals),
        'best_bs': np.quantile(bs_vals, 0.5)
    }

for i, params in results.items():
    print(f"{i} - Best mu: {params['best_mu']}, Best bs: {params['best_bs']}")

NameError: ignored